# PocketCoffea: a configuration layer for CMS analysis with Coffea

PocketCoffea is an analysis framework to perform analysis with CMS data in the NanoAOD format with a configuration scheme built on top of the Coffea package. All the usual operations of CMS analyses are shared in a common base code, including the creation of datasets, the application of selection cuts, the event reweighting, the computation of systematic uncertainty and the production of Data/MC plots.
In this tutorial, the main features of the package are presented.
A toy $Z\rightarrow\mu\mu$ analysis is implemented, using the CMS Opendata datasets from the 2012 data at $\sqrt{s}= $ 8 TeV.

Disclaimer: For demonstration purposes, we will apply corrections to MC based on the Run-2 datasets to this older datasets.

In [1]:
import os
import pocket_coffea
from pocket_coffea.utils.dataset import build_datasets
from pocket_coffea.utils.run import IterativeRunner, FuturesRunner
from pocket_coffea.utils.configurator import Configurator
from pocket_coffea.parameters.histograms import muon_hists
print(f"Available CPUs = {os.cpu_count()}")

Available CPUs = 72


## Define the analysis processor

The core of the analysis is the `processor` object. This is an object defined in `Coffea` and it defines the function that will be called taking all the events as input.

In PocketCoffea, all processors can be derived from an abstract base processor class called `BaseProcessorABC`, that is a `Coffea` processor with a well defined structure:

- **Skim events**: apply lumi mask, triggers and a loose selection to reduce the number of events processed in the following analysis steps
- **Object preselection**: define the "good" object on which will be performed the final analysis
- **Event preselection**: define the baseline selection of the analysis
- **Categorization**: split events passing the preselection into different categories (e.g. signal + control region)
- **Fill histograms** in all categories

Let's define a new processor inherited from the `BaseProcessorABC`. We don't need to rewrite the whole analysis processor, but only the methods that we want to override for our specific application:

In [2]:
import awkward as ak
from coffea.analysis_tools import PackedSelection
from pocket_coffea.workflows.base import BaseProcessorABC
from pocket_coffea.lib.objects import (
    lepton_selection,
    get_dilepton,
)


class ZmumuBaseProcessor(BaseProcessorABC):
    def __init__(self, cfg: Configurator):
        super().__init__(cfg)

    # Since the CMS opendata NanoAOD file does not have the 'genWeight' branch, we set it to 1 for each event in this demo
    def load_metadata_extra(self):
        if self._isMC:
            self.events["genWeight"] = ak.ones_like(self.events.event, dtype=float)

    # Here we customize the skim_events function to make it compatible with the 2012 Opendata dataset
    def skim_events(self):
        self._skim_masks = PackedSelection()

        if self._isMC:
            for skim_func in self._skim:
                # Apply the skim function and add it to the mask
                mask = skim_func.get_mask(
                    self.events,
                    processor_params=self.params,
                    year=self._year,
                    sample=self._sample,
                    isMC=self._isMC,
                )
                self._skim_masks.add(skim_func.id, mask)
        else:
            for skim_func in self._skim:
                # Apply the skim function and add it to the mask
                try:
                    mask = skim_func.get_mask(
                        self.events,
                        processor_params=self.params,
                        year=self._year,
                        sample=self._sample,
                        isMC=self._isMC,
                    )
                    self._skim_masks.add(skim_func.id, mask)
                except:
                    print(f"The skim function {skim_func.name} is not applied to the sample {self._sample}")
                    continue

        self.events = self.events[self._skim_masks.all(*self._skim_masks.names)]
        self.nEvents_after_skim = self.nevents
        self.output['cutflow']['skim'][self._dataset] = self.nEvents_after_skim
        self.has_events = self.nEvents_after_skim > 0

    # Since there is no jet collection, we don't need to loop over shape variations
    def get_shape_variations(self):
        '''
        Dummy generator for shape variations.
        '''
        yield "nominal"
        return
    
    def apply_object_preselection(self, variation):
        '''
        The object preselection cleans the following collections:
          - Muons
        In addition, a dilepton object is built.
        '''
        # Build masks for selection of muons
        cuts = self.params.object_preselection["Muon"]

        # Requirements on pT and eta
        passes_eta = abs(self.events.Muon.eta) < cuts["eta"]
        passes_pt = self.events.Muon.pt > cuts["pt"]
    
        # Requirements on isolation
        passes_iso = self.events.Muon.pfRelIso04_all < cuts["iso"]

        good_muons = passes_eta & passes_pt & passes_iso #& passes_id

        self.events["MuonGood"] = self.events.Muon[good_muons]

    def count_objects(self, variation):
        self.events["nMuonGood"] = ak.num(self.events.MuonGood)

## Define analysis parameters

The parameters scheme is based on composable YAML parameter files managed with the `OmegaConf` package. A nice set of defaults is defined in `pocket_coffea.parameters.defaults`, containing the most common parameters for a Run-2 CMS analysis. These include the event flags, luminosity, triggers, object preselection, plotting style etc.
The user can then customize the parameters by defining their own parameters in the form of YAML files.
The default parameters can be eventually overridden with custom parameters, thanks to the `merge_parameters_from_files()` function:

In [3]:
from pocket_coffea.parameters import defaults

localdir = os.getcwd()
default_parameters = defaults.get_default_parameters()
parameters = defaults.merge_parameters_from_files(default_parameters,
                                                  f"{localdir}/params/event_flags.yaml",
                                                  f"{localdir}/params/lumi.yaml",
                                                  f"{localdir}/params/object_preselection_semileptonic.yaml",
                                                  f"{localdir}/params/triggers.yaml",
                                                  f"{localdir}/params/plotting_style.yaml",
                                                  update=True)
parameters.keys()

dict_keys(['pileupJSONfiles', 'event_flags', 'event_flags_data', 'lumi', 'default_jets_calibration', 'jets_calibration', 'jet_scale_factors', 'btagging', 'lepton_scale_factors', 'systematic_variations', 'plotting_style', 'object_preselection', 'HLT_triggers'])

In [4]:
parameters

{'pileupJSONfiles': {'2016_PreVFP': {'file': '/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/LUM/2016preVFP_UL/puWeights.json.gz', 'name': 'Collisions16_UltraLegacy_goldenJSON'}, '2016_PostVFP': {'file': '/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/LUM/2016postVFP_UL/puWeights.json.gz', 'name': 'Collisions16_UltraLegacy_goldenJSON'}, '2017': {'file': '/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/LUM/2017_UL/puWeights.json.gz', 'name': 'Collisions17_UltraLegacy_goldenJSON'}, '2018': {'file': '/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/LUM/2018_UL/puWeights.json.gz', 'name': 'Collisions18_UltraLegacy_goldenJSON'}}, 'event_flags': {'2018': ['goodVertices', 'globalSuperTightHalo2016Filter', 'HBHENoiseFilter', 'HBHENoiseIsoFilter', 'EcalDeadCellTriggerPrimitiveFilter', 'BadPFMuonFilter'], '2017': ['goodVertices', 'globalSuperTightHalo2016Filter', 'HBHENoiseFilter', 'HBHENoiseIsoFilter', 'EcalDeadCellTriggerPrimitiveFilter', 'B

## Define datasets

A dictionary containing the datasets information is passed to the `Configurator`. This includes the list of json datasets, together with a dictionary to filter the datasets by the sample key or by year. At this point it is also possible to define **subsamples**, i.e. split samples into different subsets (e.g. split data by the trigger path or split a MC sample into distinct components).

In [17]:
datasets = {
    "jsons": [f"{localdir}/datasets/DYJetsToLL.json",
              f"{localdir}/datasets/DATA_DoubleMuon.json"],
    "filter" : {
        "samples": ["DYJetsToLL",
                    "DATA_DoubleMuon"
                   ],
        "samples_exclude" : [],
        "year": ["2012"]
    },
    "subsamples" : {},
}

## Skim, preselection and categorization

All the selection operations are performed by making use of `Cut` objects. These objects are defined by a function of the events and a set of parameters. This way, for a given function we can have multiple cuts by changing the parameters that we pass to the `Cut` constructor.
Below is an example of a cut function `dimuon()` that is used to build a `Cut` object named `dimuon_presel`:

In [6]:
from pocket_coffea.lib.cut_definition import Cut
from pocket_coffea.lib.cut_functions import get_nObj_min, get_nObj_eq, get_HLTsel
from pocket_coffea.parameters.cuts import passthrough

def dimuon(events, params, year, sample, **kwargs):

    # Masks for same-flavor (SF) and opposite-sign (OS)
    SF = ((events.nMuonGood == 2) & (events.nElectronGood == 0))
    OS = events.ll.charge == 0

    mask = (
        (events.nLeptonGood == 2)
        & (ak.firsts(events.MuonGood.pt) > params["pt_leading_muon"])
        & OS & SF
        & (events.ll.mass > params["mll"]["low"])
        & (events.ll.mass < params["mll"]["high"])
    )

    # Pad None values with False
    return ak.where(ak.is_none(mask), False, mask)

dimuon_presel = Cut(
    name="dilepton",
    params={
        "pt_leading_muon": 25,
        "mll": {'low': 25, 'high': 2000},
    },
    function=dimuon,
)


## Weights and systematic variations

Another important ingredient of our analyses is the application of correction to the MC simulation and the proper accounting of systematic uncertainties. In PocketCoffea, a dedicated module `WeightsManager` is handling the application of weights to MC. In the configuration file, the user just needs to list the names of the correction weights to apply and the systematic variations that have to be stored in the output histograms.

In [7]:
weights = {
    "common": {
        "inclusive": ["genWeight","lumi","XS",
                      #"pileup",
                      #"sf_ele_reco", "sf_ele_id",
                      #"sf_mu_id","sf_mu_iso",
                      ],
        "bycategory" : {
        }
    },
    "bysample": {
    }
}

variations = {
    "weights": {
        "common": {
            "inclusive": [ ],#"pileup" ],
            "bycategory" : {
            }
        },
        "bysample": {
        }
    },
    "shape": {
        "common":{
            "inclusive": [ ]
        }
    }
}

In [8]:
cfg = Configurator(
    parameters=parameters,
    datasets=datasets,
    workflow=ZmumuBaseProcessor,
    skim = [get_nObj_min(2, 15., "Muon"),
            get_HLTsel(primaryDatasets=["DoubleMuon"])],
    preselections = [get_nObj_min(2, 25., "MuonGood")],
    categories = {
        "inclusive" : [passthrough],
        "2mu" : [get_nObj_eq(2, coll="MuonGood")],
        "3mu" : [get_nObj_eq(3, coll="MuonGood")],
    },
    weights = weights,
    variations = variations,
    variables = {
        **muon_hists(coll="MuonGood"),
        **muon_hists(coll="MuonGood", pos=0),
    }
)
#cfg.filter_dataset(nfiles=1)
output_dir = "output"
#cfg.save_config(output_dir)

In [9]:
run_options = {
    "executor"       : "local",
    "env"            : "conda",
    "workers"        : 1,
    "scaleout"       : 32,
    "worker_image"   : "/cvmfs/unpacked.cern.ch/gitlab-registry.cern.ch/cms-analysis/general/pocketcoffea:lxplus-cc7-latest",
    "queue"          : "standard",
    "walltime"       : "02:00:00",
    "mem_per_worker" : "4GB", # GB
    "disk_per_worker" : "1GB", # GB
    "exclusive"      : False,
    "chunk"          : 200000,
    "retries"        : 50,
    "treereduction"  : 20,
    "adapt"          : False,
}
runner = FuturesRunner(architecture="local", run_options=run_options, output_dir=output_dir, loglevel="INFO")

cp: ‘/t3home/mmarcheg/.x509up_u718’ and ‘/t3home/mmarcheg/.x509up_u718’ are the same file


In [10]:
filesets = {
    "DYJetsToLL_2012": {
        "metadata": {
            "sample": "DYJetsToLL",
            "year": "2012",
            "isMC": True,
            "xsec": 4048.42,
        },
    "files": [f"{localdir}/datasets/DYJetsToLL.root"]
    },
    "DATA_DoubleMuon_2012": {
        "metadata": {
            "sample": "DATA_DoubleMuon",
            "year": "2012",
            "isMC": False,
	    "primaryDataset": "DoubleMuon",
	    "era": "C"
        },
    "files": [f"{localdir}/datasets/Run2012B_DoubleMuParked.root"]
    }
}
key_data = "DATA_DoubleMuon_2012"
filesets_data = {key_data : filesets[key_data]}

In [11]:
runner.run(
    cfg.filesets,
    cfg.processor_instance,
    full=True,
)

[INFO    ] Working on samples: ['DYJetsToLL_2012', 'DATA_DoubleMuon_2012']


Output()

Output()

/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_jetIdx => Jet
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_photonIdx => Photon
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx1 => SubJet
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx2 => SubJet
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference

The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuonThe skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon

The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon
The skim function HLT_trigger_DoubleMuon is not applie

...

The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon


/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_genPartIdx => GenPart
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for FatJet_genJetAK8Idx => GenJetAK8
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for GenPart_genPartIdxMother => GenPart
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for GenVisTau_genPartIdxMother => GenPart
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWa

The skim function HLT_trigger_DoubleMuon is not applied to the sample DATA_DoubleMuon


/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for Electron_genPartIdx => GenPart
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for FatJet_genJetAK8Idx => GenJetAK8
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for GenPart_genPartIdxMother => GenPart
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWarning: Missing cross-reference index for GenVisTau_genPartIdxMother => GenPart
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:201: RuntimeWa

Saving output to output/output_all.coffea


In [13]:
os.listdir("output")

['parameters_dump.yaml',
 'configurator.pkl',
 'output_all.coffea',
 'config.json']

## Produce Data/MC plots

In [14]:
from coffea.util import load
from pocket_coffea.utils.plot_utils import PlotManager, Shape
o = load("output/output_all.coffea")
o.keys()

dict_keys(['sum_genweights', 'sumw', 'cutflow', 'variables', 'columns', 'processing_metadata', 'datasets_metadata'])

In [15]:
plotter = PlotManager(
    variables=o["variables"].keys(),
    hist_objs=o["variables"],
    datasets_metadata=o['datasets_metadata'],
    plot_dir="plots",
    style_cfg=parameters['plotting_style'],
    only_cat=None,
    workers=8,
    log=False,
    density=False,
    save=True
)
plotter.plot_datamc_all(syst=True, spliteras=False)

Plotting:  MuonGood_eta_2012
Plotting:  MuonGood_phi_1_2012
Plotting: Plotting:  Plotting:  MuonGood_pt_1_2012 MuonGood_eta_1_2012
MuonGood_pt_2012

Plotting:  MuonGood_phi_2012


/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:474: RuntimeWarning: invalid value encountered in divide
  ratio = num / den
/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:477: RuntimeWarning: invalid value encountered in divide
  ratio_unc = np.sqrt(num) / den
/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:838: RuntimeWarning: invalid value encountered in divide
  return np.where(self.nominal != 0, self.up / self.nominal, 1)
/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:842: RuntimeWarning: invalid value encountered in divide
  return np.where(self.nominal != 0, self.down / self.nominal, 1)


Saving plots/2mu/MuonGood_eta_2012_2mu.png

/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:474: RuntimeWarning: invalid value encountered in divide
  ratio = num / den


/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:477: RuntimeWarning: invalid value encountered in divide
  ratio_unc = np.sqrt(num) / den
/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:474: RuntimeWarning: invalid value encountered in divide
  ratio = num / den
/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:477: RuntimeWarning: invalid value encountered in divide
  ratio_unc = np.sqrt(num) / den
/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:838: RuntimeWarning: invalid value encountered in divide
  return np.where(self.nominal != 0, self.up / self.nominal, 1)


...

Saving plots/2mu/MuonGood_eta_1_2012_2mu.png


/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:842: RuntimeWarning: invalid value encountered in divide
  return np.where(self.nominal != 0, self.down / self.nominal, 1)


Saving plots/2mu/MuonGood_pt_2012_2mu.png
Saving plots/2mu/MuonGood_phi_2012_2mu.png
Saving plots/3mu/MuonGood_eta_2012_3mu.png
Saving plots/3mu/MuonGood_eta_1_2012_3mu.png


/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:474: RuntimeWarning: divide by zero encountered in divide
  ratio = num / den
/work/mmarcheg/PocketCoffea/pocket_coffea/utils/plot_utils.py:477: RuntimeWarning: divide by zero encountered in divide
  ratio_unc = np.sqrt(num) / den
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/matplotlib/axes/_axes.py:3680: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err


Saving plots/3mu/MuonGood_pt_2012_3mu.png
Saving plots/3mu/MuonGood_pt_1_2012_3mu.png
Saving plots/3mu/MuonGood_phi_1_2012_3mu.png
Saving plots/3mu/MuonGood_phi_2012_3mu.png
Saving plots/inclusive/MuonGood_eta_2012_inclusive.png
Saving plots/inclusive/MuonGood_pt_2012_inclusive.png
Saving plots/inclusive/MuonGood_pt_1_2012_inclusive.png
Saving plots/inclusive/MuonGood_phi_1_2012_inclusive.png
Saving plots/inclusive/MuonGood_eta_1_2012_inclusive.png
Saving plots/inclusive/MuonGood_phi_2012_inclusive.png


## Fetch the dataset from DAS and rucio

In [16]:
build_datasets(
    "datasets/datasets_definitions.json",
    overwrite=True,
    #local_prefix=self.local_prefix,
    #whitelist_sites=self.whitelist_sites,
    #blacklist_sites=self.blacklist_sites,
    #regex_sites=self.regex_sites,
    parallelize=4,
)

****************************************************************************************************************************************************************



> Working on dataset: > Working on dataset:  > Working on dataset: > Working on dataset:  DATA_DoubleMuonDATA_SingleEle  

>> Query for sample: DATA_DoubleMu,  das_name: ['/DoubleMuon/Run2017B-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD']TTTo2L2NuTTbbSemiLeptonic_4f>> Query for sample: DATA_SingleEle,  das_name: ['/SingleElectron/Run2016B-ver1_HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD', '/SingleElectron/Run2016B-ver2_HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD']



>> Query for sample: TTTo2L2Nu,  das_name: ['/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL16NanoAODAPVv9-106X_mcRun2_asymptotic_preVFP_v11-v1/NANOAODSIM']>> Query for sample: TTbbSemiLeptonic,  das_name: ['/TTbb_4f_TTToSemiLeptonic_TuneCP5-Powheg-Openloops-Pythia8/RunIISummer20UL16NanoAODAPVv9-106X_mcRun2_asymptotic_preVFP_v11-v1/NANOAODSIM']

dict_keys(['DATA

/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsweb.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsweb.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsweb.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

>> Query for sample: DATA_DoubleMu,  das_name: ['/DoubleMuon/Run2017C-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD']
>> Query for sample: TTbbSemiLeptonic,  das_name: ['/TTbb_4f_TTToSemiLeptonic_TuneCP5-Powheg-Openloops-Pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM']
>> Query for sample: TTTo2L2Nu,  das_name: ['/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL16NanoAODv9-106X_mcRun2_asymptotic_v17-v1/NANOAODSIM']


/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsweb.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsweb.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsweb.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

>> Query for sample: DATA_DoubleMu,  das_name: ['/DoubleMuon/Run2017D-UL2017_MiniAODv2_NanoAODv9-v1/NANOAOD']
>> Query for sample: TTbbSemiLeptonic,  das_name: ['/TTbb_4f_TTToSemiLeptonic_TuneCP5-Powheg-Openloops-Pythia8/RunIISummer20UL17NanoAODv9-106X_mc2017_realistic_v9-v1/NANOAODSIM']
>> Query for sample: TTTo2L2Nu,  das_name: ['/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL17NanoAODv9-106X_mc2017_realistic_v9-v1/NANOAODSIM']
>> Query for sample: DATA_SingleEle,  das_name: ['/SingleElectron/Run2016C-HIPM_UL2016_MiniAODv2_NanoAODv9-v2/NANOAOD']


...

>> Query for sample: DATA_SingleMuon,  das_name: ['/SingleMuon/Run2018C-UL2018_MiniAODv2_NanoAODv9-v2/NANOAOD']


/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsweb.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


>> Query for sample: DATA_SingleMuon,  das_name: ['/SingleMuon/Run2018D-UL2018_MiniAODv2_NanoAODv9-v1/NANOAOD']


/t3home/mmarcheg/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cmsweb.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Saving datasets DATA_DoubleMuon to datasets/DATA_DoubleMuon.json
Saving datasets DATA_DoubleMuon to datasets/DATA_DoubleMuon_redirector.json
Saving datasets DATA_SingleMuon to datasets/DATA_SingleMuon.json
Saving datasets DATA_SingleMuon to datasets/DATA_SingleMuon_redirector.json
Saving datasets DATA_SingleEle to datasets/DATA_SingleEle.json
Saving datasets DATA_SingleEle to datasets/DATA_SingleEle_redirector.json
Saving datasets ttHTobb to datasets/signal_ttHTobb.json
Saving datasets ttHTobb to datasets/signal_ttHTobb_redirector.json
Saving datasets TTbbSemiLeptonic_4f to datasets/backgrounds_MC_TTbb.json
Saving datasets TTbbSemiLeptonic_4f to datasets/backgrounds_MC_TTbb_redirector.json
Saving datasets TTToSemiLeptonic to datasets/backgrounds_MC_ttbar.json
Saving datasets TTToSemiLeptonic to datasets/backgrounds_MC_ttbar_redirector.json
Saving datasets TTTo2L2Nu to datasets/backgrounds_MC_ttbar.json
Saving datasets TTTo2L2Nu to datasets/backgrounds_MC_ttbar_redirector.json
Saving da